# Documentation snippets
## Quick start

```shell
pip install py-llm-core

#: To use OpenAI models, set your API key
export OPENAI_API_KEY=sk-<replace with your actual api key>

#: To use MistralAI models, set your API key
export MISTRAL_API_KEY=sk-<replace with your actual api key>

#: To use local models (i.e. completely offline),
#: download and store your models in ~/.cache/py-llm-core/models/
#: You may change this directory by setting the environment variable MODELS_CACHE_DIR
#: See the src/settings.py module

#: The following commands download the best models (you can use any GGUF models)
#: LLaMA-3.1-8B (Quantized version Q4_K_M)
#: Mistral 7B v0.3 (Quantized version Q4_K_M)

mkdir -p ~/.cache/py-llm-core/models
wget -O ~/.cache/py-llm-core/models/llama-8b-3.1-q4 \
    https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf?download=true

wget -O ~/.cache/py-llm-core/models/mistral-7b-v0.3-q4 \
    https://huggingface.co/lmstudio-community/Mistral-7B-Instruct-v0.3-GGUF/resolve/main/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf?download=true
```

In [1]:
from dataclasses import dataclass

@dataclass
class Book:
    title: str
    summary: str
    author: str
    published_year: int

@dataclass
class BookCollection:
    books: list[Book]


text = """The Foundation series is a science fiction book series written by
American author Isaac Asimov. First published as a series of short
stories and novellas in 1942–50, and subsequently in three books in
1951–53, for nearly thirty years the series was widely known as The
Foundation Trilogy: Foundation (1951), Foundation and Empire (1952),
and Second Foundation (1953). It won the one-time Hugo Award for "Best
All-Time Series" in 1966. Asimov later added new volumes, with two
sequels, Foundation's Edge (1982) and Foundation and Earth (1986), and
two prequels, Prelude to Foundation (1988) and Forward the Foundation
(1993).

The premise of the stories is that in the waning days of a future
Galactic Empire, the mathematician Hari Seldon devises the theory of
psychohistory, a new and effective mathematics of sociology. Using
statistical laws of mass action, it can predict the future of large
populations. Seldon foresees the imminent fall of the Empire, which
encompasses the entire Milky Way, and a dark age lasting 30,000 years
before a second empire arises. Although the momentum of the Empire's
fall is too great to stop, Seldon devises a plan by which "the
onrushing mass of events must be deflected just a little" to
eventually limit this interregnum to just one thousand years. The
books describe some of the dramatic events of those years as they are
shaped by the underlying political and social mechanics of Seldon's
Plan.
"""


In [2]:
from llm_core.parsers import OpenWeightsParser

# default model is "mistral-7b-v0.3-q4"
with OpenWeightsParser(BookCollection) as parser:
    books_collection = parser.parse(text)

    for book in books_collection.books:
        print(book)

Book(title='Foundation', summary="The first book in the series, published in 1951, follows the early stages of the Foundation and the implementation of Hari Seldon's psychohistory.", author='Isaac Asimov', published_year=1951)
Book(title='Foundation and Empire', summary='The second book, published in 1952, continues the story of the Foundation and introduces new characters and conflicts.', author='Isaac Asimov', published_year=1952)
Book(title='Second Foundation', summary='The third book, published in 1953, focuses on the Second Foundation and its role in the events of the series.', author='Isaac Asimov', published_year=1953)
Book(title="Foundation's Edge", summary='The fourth book, published in 1982, is a sequel to the original trilogy and follows the Foundation as it faces new challenges.', author='Isaac Asimov', published_year=1982)
Book(title='Foundation and Earth', summary='The fifth book, published in 1986, is another sequel to the original trilogy and resolves some of the myster

In [3]:
from llm_core.parsers import OpenAIParser

# default model is "gpt-4o-mini"
with OpenAIParser(BookCollection) as parser:
    books_collection = parser.parse(text)
    print(len(books_collection.books))
    for book in books_collection.books:
        print(book)

7
Book(title='Foundation', summary="The first book in the Foundation series, it introduces the concept of psychohistory and the mathematician Hari Seldon's plan to save civilization from a dark age.", author='Isaac Asimov', published_year=1951)
Book(title='Foundation and Empire', summary='The second book in the Foundation series, it continues to explore the challenges faced by the Foundation as it tries to navigate the decline of the Galactic Empire.', author='Isaac Asimov', published_year=1952)
Book(title='Second Foundation', summary="The third book in the Foundation series, it reveals the hidden Second Foundation and its role in Seldon's plan to preserve knowledge and culture during the dark age.", author='Isaac Asimov', published_year=1953)
Book(title="Foundation's Edge", summary="A sequel to the original trilogy, this book follows the search for the mythical Second Foundation and the implications of Seldon's plan.", author='Isaac Asimov', published_year=1982)
Book(title='Foundation

In [4]:
from llm_core.parsers import MistralAIParser

# default model is "open-mistral-nemo"
with MistralAIParser(BookCollection) as parser:
    books_collection = parser.parse(text)
    print(len(books_collection.books))
    for book in books_collection.books:
        print(book)

7
Book(title='Foundation', summary="Foundation is the first book in the Foundation series, published in 1951. It introduces the concept of psychohistory and Hari Seldon's plan to shorten the interregnum between the Galactic Empire's fall and the rise of the Second Empire.", author='Isaac Asimov', published_year=1951)
Book(title='Foundation and Empire', summary="Foundation and Empire is the second book in the Foundation series, published in 1952. It continues the story of the Foundation's growth and the implementation of Hari Seldon's plan.", author='Isaac Asimov', published_year=1952)
Book(title='Second Foundation', summary="Second Foundation is the third book in the Foundation series, published in 1953. It reveals the existence of the Second Foundation and the final stages of Hari Seldon's plan.", author='Isaac Asimov', published_year=1953)
Book(title="Foundation's Edge", summary="Foundation's Edge is the fourth book in the Foundation series, published in 1982. It is set 120 years aft

In [5]:
import requests
from decouple import config
from dataclasses import dataclass

from llm_core.llm import OpenAIChatModel


@dataclass
class WebSearchProvider:
    query: str

    def __call__(self):
        url = "https://api.search.brave.com/res/v1/web/search"
        headers = {"X-Subscription-Token": config("BRAVE_AI_API_KEY")}
        response = requests.get(
            url,
            headers=headers,
            params={
                "q": self.query,
                "extra_snippets": True
            }
        )
        return response.json()["web"]["results"][0:5]


providers = [WebSearchProvider]

llm = OpenAIChatModel(name="gpt-4o-mini")
resp = llm.ask(
    prompt="Who won the 400m men individual medley at the 2024 Olympics?",
    tools=providers
)

print(resp.choices[0].message.content)

Leon Marchand won the gold medal in the men's 400m individual medley at the 2024 Olympics.


In [6]:
import hashlib
from enum import Enum
from dataclasses import dataclass
from llm_core.assistants import OpenAIAssistant


HashFunction = Enum("HashFunction", ["sha512", "sha256", "md5"])


@dataclass
class HashProvider:
    hash_function: HashFunction
    content: str

    def __call__(self):
        hash_fn = getattr(hashlib, self.hash_function.name)
        return hash_fn(self.content.encode('utf-8')).hexdigest()


@dataclass
class Hash:
    system_prompt = "You are a helpful assistant"
    prompt = "{prompt}"

    hashed_content: str
    hash_algorithm: HashFunction
    hash_value: str

    @classmethod
    def ask(cls, prompt):
        with OpenAIAssistant(cls, model="gpt-4o-mini") as assistant:
            assistant.tools = [HashProvider]
            response = assistant.process(prompt=prompt)
            return response

Hash.ask('Compute the md5 for `py-llm-core`')

Hash(hashed_content='py-llm-core', hash_algorithm=<HashFunction.md5: 3>, hash_value='d187ea11586be0b9e32b09c3021eb021')

In [7]:
from llm_core.splitters import TokenSplitter
import codecs

text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""

# You can encode the text into tokens like that:
tokens = codecs.encode(text, 'tiktoken')
token_length = len(tokens)

# Chunking and splitting
splitter = TokenSplitter(
    chunk_size=50,
    chunk_overlap=0
)

for chunk in splitter.chunkify(text):
    print(chunk)

Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a
 single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.



In [8]:
from enum import Enum
from dataclasses import dataclass
from llm_core.assistants import OpenWeightsAssistant

class TargetItem(Enum):
    PROJECT = 1
    TASK = 2
    COMMENT = 3
    MEETING = 4


class CRUDOperation(Enum):
    CREATE = 1
    READ = 2
    UPDATE = 3
    DELETE = 4


@dataclass
class UserQuery:
    system_prompt = "You are a helpful assistant."
    prompt = """
    Analyze the user's query and convert his intent to:
    - an operation (among CRUD)
    - a target item

    Query: {prompt}
    """
    operation: CRUDOperation
    target: TargetItem


def ask(prompt):
    with OpenWeightsAssistant(UserQuery, model="mistral-7b-v0.3-q4") as assistant:
        user_query = assistant.process(prompt=prompt)
        return user_query

ask('Cancel all my meetings for the week')

UserQuery(operation=<CRUDOperation.DELETE: 4>, target=<TargetItem.MEETING: 4>)